# Simple example on using Instrumental Variables method for estimation

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import pandas as pd
import patsy as ps

from statsmodels.sandbox.regression.gmm import IV2SLS
import os, sys
from dowhy import CausalModel

## Loading the dataset

We create a fictitious dataset with the goal of estimating the impact of education on future earnings of an individual. The `ability`  of the individual is a confounder and being given an `education_voucher` is the instrument.

In [3]:
n_points = 1000
education_abilty = 1
education_voucher = 2
income_abilty = 2
income_education = 4


# confounder
ability = np.random.normal(0, 3, size=n_points)

# instrument
voucher = np.random.normal(2, 1, size=n_points) 

# treatment
education = np.random.normal(5, 1, size=n_points) + education_abilty * ability +\
            education_voucher * voucher

# outcome
income = np.random.normal(10, 3, size=n_points) +\
         income_abilty * ability + income_education * education

# build dataset (exclude confounder `ability` which we assume to be unobserved)
data = np.stack([education, income, voucher]).T
df = pd.DataFrame(data, columns = ['education', 'income', 'voucher'])

In [5]:
df

,education,income,voucher
0,4.048885,17.589657,2.206754
1,7.539867,34.356232,3.375975
2,10.181709,53.338791,2.587792
3,9.802940,45.885771,3.220738
4,4.185900,17.435756,2.993672
...,...,...,...
995,13.983056,81.673099,1.351080
996,11.760278,64.963654,2.210033
997,7.974032,44.436872,1.059596
998,8.564314,45.252911,1.542488


## Using DoWhy to estimate the causal effect of education on future income

We follow the four steps: 
1) model the problem using causal graph, 

2) identify if the causal effect can be estimated from the observed variables, 

3) estimate the effect, and 

4) check the robustness of the estimate. 

In [ ]:
#Step 1: Model
model=CausalModel(
        data = df,
        treatment='education',
        outcome='income',
        common_causes=['U'],
        instruments=['voucher']
        )
model.view_model()
from IPython.display import Image, display
display(Image(filename="causal_model.png"))

In [ ]:
# Step 2: Identify
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
# Step 3: Estimate
#Choose the second estimand: using IV
estimate = model.estimate_effect(identified_estimand,
        method_name="iv.instrumental_variable", test_significance=True)

print(estimate)

We have an estimate, indicating that increasing `education` by one unit increases `income` by 4 points.  

Next we check the robustness of the estimate using a Placebo refutation test. In this test, the treatment is replaced by an independent random variable (while preserving the correlation with the instrument), so that the true causal effect should be zero. We check if our estimator also provides the correct answer of zero. 

In [ ]:
# Step 4: Refute
ref = model.refute_estimate(identified_estimand, estimate, method_name="placebo_treatment_refuter", placebo_type="permute") # only permute placebo_type works with IV estimate
print(ref)

The refutation gives confidence that the estimate is not capturing any noise in the data.

Since this is simulated data, we also know the true causal effect is `4` (see the `income_education` parameter of the data-generating process above)

Finally, we show the same estimation by another method to verify the result from DoWhy.

In [ ]:
income_vec, endog = ps.dmatrices("income ~ education", data=df)
exog = ps.dmatrix("voucher", data=df)

m = IV2SLS(income_vec, endog, exog).fit()
m.summary()